# 1.3 데이터저장
---

| 인코딩 방식 | encoding값 | 특징 |
|---|:---:|---:|
| UTF-8 | 'utf-8' | 유니코드 문자를 사용할 수 있지만 엑셀에서 문제 발생 |
| EUC_KR | 'cp949' | 엑셀에서 문제없지만 사용할 수 있는 문자가 한정적 |


## CSV형식으로 저장

- 하나의 레코드를 한 라인에 저장하는 텍스트 포맷 파일
- 각 라인의 컬럼값은 쉼표 등의 구분자를 사용
- 데이터 내에 쉼표 등의 구분자 사용하는 문자가 포함되어 있다면 파싱 시 문제 발생
- 데이터 내에 쉼표 등의 문자가 포함되어 있는 경우 TSV형식 사용 필요

### 2.6.1.1 save_csv 실습

In [1]:
import csv

In [2]:
# 파일을 엽니다. newline=''으로 줄바꿈 코드의 자동 변환을 제어합니다.
with open('top_cities.csv', 'w', newline='', encoding='utf-8') as f:
    # csv.writer는 파일 객체를 매개변수로 지정합니다.
    writer = csv.writer(f)  
    
    # 첫 번째 줄에는 헤더를 작성
    # 두번째 매개변수에 필드명 리스트를 지정
    writer.writerow(['rank', 'city', 'population'])  
    
    # writerows()에 리스트를 전달하면 여러 개의 값을 출력합니다.
    writer.writerows([
        [1, '상하이', 24150000],
        [2, '카라치', 23500000],
        [3, '베이징', 21516000],
        [4, '텐진', 14722100],
        [5, '이스탄불', 14160467],
    ])

### 2.6.1.2 save_csv_dict 실습

In [3]:
import csv

In [5]:
with open('top_cities_dict.csv', 'w', newline='', encoding='utf-8') as f:
    # 첫 번째 매개변수에 파일 객체
    # 두 번째 매개변수에 필드 이름 리스트를 지정합니다.
    writer = csv.DictWriter(f, ['rank', 'city', 'population'])
      # 첫 번째 줄에 헤더를 입력합니다.
    writer.writeheader()
    # writerows()로 여러 개의 데이터를 딕셔너리 형태로 작성합니다.
    writer.writerows([
        {'rank': 1, 'city': '상하이', 'population': 24150000},
        {'rank': 2, 'city': '카라치', 'population': 23500000},
        {'rank': 3, 'city': '베이징', 'population': 21516000},
        {'rank': 4, 'city': '텐진', 'population': 14722100},
        {'rank': 5, 'city': '이스탄불', 'population': 14160467},
    ])

### 2.6.1.3 encoding_check 실습

In [6]:
import chardet

In [7]:
char_dic = chardet.detect(open('top_cities.csv','rb').read())

In [9]:
char_dic

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [8]:
char_dic['encoding']

'utf-8'

---

## JSON 형식으로 저장

- Key-value 형식의 데이터 객체를 저장, 전달하기 위해 텍스트 형식의 개방형 표준 포맷
- 데이터 타입
    - 문자열
    - 숫자
    - 객체(JSON Object)
    - 배열
    - 불리언

### 2.6.2.1 save_json 실습

In [11]:
import json

In [15]:
# 데이터 입력
# 리스트 형식에 dict 값을 입력
cities = [ 
{'rank': 1, 'city':'상하이', 'population': 24150000},
{'rank': 2, 'city':'카라치', 'population': 23500000},
{'rank': 3, 'city':'베이징', 'population': 21516000},
{'rank': 4, 'city':'텐진', 'population': 14722100}, 
{'rank': 5, 'city':'이스탄불', 'population':14160467},
]

In [13]:
# 쓰기 모드
# dump : 파일 시스템에 저장하기 위한 json 라이브러리 함수
with open('top_cities.json', 'w') as fw:
    json.dump(cities, fw)

In [14]:
# 읽기 모드
with open('top_cities.json', 'r') as fr:
    json_file = json.load(fr)
    print(json_file)

[{'rank': 1, 'city': '상하이', 'population': 24150000}, {'rank': 2, 'city': '카라치', 'population': 23500000}, {'rank': 3, 'city': '베이징', 'population': 21516000}, {'rank': 4, 'city': '텐진', 'population': 14722100}, {'rank': 5, 'city': '이스탄불', 'population': 14160467}]


In [21]:
type(cities)

list

In [20]:
cities[1]['city']

'카라치'

---

## SQLite3 DBMS로 저장

- 파일시스템 기반의 경량 관계형 DBMS
- 경량 관계형 DBMS로 스마트폰 등의 embededed 환경에서 널리 사용
- 대용량 데이터 및 트랜잭션 처리 부적합
- 교육 목적, 경량 데이터 및 트랜잭션 처리

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

### 2.6.3.1 save_sqlite3 실습

In [22]:
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [23]:
DB_NAME = 'top_cities.db'
TABLE_NAME = 'TOP_CITIES'

In [24]:
# db에 저장
def db_save(df, db_name, table_name):
    with sqlite3.connect(db_name) as con:
        try:
            # to_sql(name = 테이블 이름, con = 객체에 대한 레거시 제공,index=DateFrame 인덱스를 열로 쓸것인지,
            #        if_exists='fail, replace,append')
            df.to_sql(name=table_name, con = con, index=False, if_exists='append')
            # if_exists : {'fail', 'replace','append'} default : fail
                # fail : ValueError를 발생
                # replace : 새 값을 삽입하기 전에 테이블 삭제
                # append : 기존 테이블에 새 값을 삽입
        
        except Exception as e:
            print(str(e))
        print(len(df),'건 저장완료..')

In [25]:
# db 출력
def db_select(db_name, table_name):
    with sqlite3.connect(db_name) as con:
        try:
            query = "SELECT * FROM {}".format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e))
        return df

In [26]:
# db 삭제
def db_delete(db_name, table_name):
    with sqlite3.connect(db_name) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM {}'.format(table_name)
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [27]:
top_cites = pd.read_csv('top_cities.csv')
db_save(top_cites, DB_NAME, TABLE_NAME)

5 건 저장완료..


In [28]:
df = db_select(DB_NAME, TABLE_NAME)
df

,rank,city,population
0,1,상하이,24150000
1,2,카라치,23500000
2,3,베이징,21516000
3,4,텐진,14722100
4,5,이스탄불,14160467
